In [1]:
from typing import List
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, validator
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import os

In [2]:
model = OpenAI()

In [3]:
class Ticket(BaseModel):
    date: str = Field(description="show date")
    time: str = Field(description="show time")
    theater: str = Field(description="theater name")
    count: int = Field(description="number of tickets")
    movie: str = Field(description="preferred movie")

In [4]:
parser = PydanticOutputParser(pydantic_object=Ticket)

In [5]:
ticket_template = '''
  Book us a movie ticket for two this Friday at 6:00 PM. 
  Choose any theater, it doesn't matter. Send the confirmation by email.
  Our preferred movie is: {query}
  Format instructions:
  {format_instructions}
'''

In [7]:
prompt = PromptTemplate(
    template=ticket_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [8]:
input = prompt.format_prompt(query="Interstellar")

In [9]:
print(input.to_string())


  Book us a movie ticket for two this Friday at 6:00 PM. 
  Choose any theater, it doesn't matter. Send the confirmation by email.
  Our preferred movie is: Interstellar
  Format instructions:
  The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"date": {"description": "show date", "title": "Date", "type": "string"}, "time": {"description": "show time", "title": "Time", "type": "string"}, "theater": {"description": "theater name", "title": "Theater", "type": "string"}, "count": {"description": "number of tickets", "title": "Count", "type": "integer"}, "movie": {"description": 

In [11]:
output = model.invoke(input.to_string())

In [12]:
parsed_response = parser.parse(output)
print(parsed_response)

date='Friday' time='6:00 PM' theater='Any' count=2 movie='Interstellar'


In [13]:
type(parsed_response)

__main__.Ticket